# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
%pip install openai httpx==0.28.1

In [ ]:
import pandas as pd
from google.colab import userdata
from IPython.display import display

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [ ]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='url_to_openai_gpt_api/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key=userdata.get('your_openai_key')
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [ ]:
# Code example of reading input data

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = 'xxx'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [ ]:
# Code example of generating output document

# Creates a new shared Google Worksheet every invocation with the proper structure
# Note: This code should be executed from the google colab once you are ready, it will not work locally
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# IMPORTANT: You need to authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o
auth.authenticate_user()
creds, _ = default()

gc = gspread.authorize(creds)

# This code goes after creating google client
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.gspread.org/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/1K9yuCpqTY5lHlWHmq1ui2Ah7PZznHlWz4tqnbZgvndY


In [ ]:
output_document.del_worksheet(output_document.worksheet("Sheet1")) # Delete empty, useless sheet called "Sheet1"

{'spreadsheetId': '1K9yuCpqTY5lHlWHmq1ui2Ah7PZznHlWz4tqnbZgvndY',
 'replies': [{}]}

In [ ]:
GPT_MODEL = "gpt-4o" # Set gpt version constant

# Task 1. Classify emails

In [ ]:
def classify_emails(emails):
  """
  Classifies emails in the DataFrame as 'order request' or 'product inquiry'
  by sending them to the ChatGPT API for processing.
  Returns a Dataframe
  """

  prompt = (
        "You are a highly accurate classification assistant. "
        "Your task is to classify each email into one of the following two categories:\n\n"
        "1. order request – Emails where the sender expresses an intent to place an order or make a purchase. "
        "This includes informal phrases like 'I'd want to order', 'I'd like to get', 'Can I have', 'Please send me', etc. "
        "Even if no product details or quantities are specified, if the sender clearly wants to buy something, it's an order request.\n\n"
        "If the sender indicates a later date for ordering, it is an order request.\n\n"
        "2. product inquiry – Emails where the sender is asking for product details, specifications, availability, or pricing, "
        "without clearly indicating they want to place an order.\n\n"
        "**Important guidelines:**\n"
        "- If an email contains *both* an inquiry and an intent to order, classify it as **order request**.\n"
        "- Do not rely on formality: casual language like 'I’d want to order' still counts as an **order request**.\n"
        "- If the email is ambiguous, choose the most dominant intent.\n\n"
        "- If the email has no subject, use only the content of the email message."
        "Explain the reasoning in why you classified each mail as such."
        "Respond only with the format:\n'Email ID::category::Explaination\n\n"
        "Emails will be provided by user."
    )

  # Send data to the ChatGPT API
  response = client.chat.completions.create(
      model=GPT_MODEL,
      messages=[
          {"role": "system", "content": prompt},
           {"role": "user", "content": f"Email Data:\n{emails}"}
          ],
      temperature=0
      )

  returned_data = response.choices[0].message.content.strip().splitlines() # Clean response content and split lines
  cleaned_data = [entry.strip() for entry in returned_data if entry] # Clean each line

  return cleaned_data # Return cleaned data

tmp_data = classify_emails(emails_df.to_dict(orient='records')) # Run classification

In [ ]:
classifications = pd.DataFrame([e.split('::')[:-1] for e in tmp_data], columns=['Email ID', 'category']) # Only retrieve the email ID and the category from previous results
email_classification_sheet.update([classifications.columns.values.tolist()] + classifications.values.tolist()) # Update Sheet with required resutls

{'spreadsheetId': '1K9yuCpqTY5lHlWHmq1ui2Ah7PZznHlWz4tqnbZgvndY',
 'updatedRange': "'email-classification'!A1:B24",
 'updatedRows': 24,
 'updatedColumns': 2,
 'updatedCells': 48}

# Task 2. Process order requests

In [ ]:
# Install langchain and chroma
!pip install langchain-core langchain-chroma langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 23.0 MB/s eta 0:

In [ ]:
# Necessary imports
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import chromadb

In [ ]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', api_key=userdata.get('openai_key'), base_url='url_to_openai_gpt_api/v1/') # Instantiate embedding model

In [ ]:
def index_product_catalog(products: pd.DataFrame, embedding_model) -> Chroma:
  """
  Index the product catalog into a Chroma vector store. This will serve for RAG similarity searches with vectors, in order to limit the product entries provided to gpt.
  :param products: Dataframe containing the catalog.
  :param embedding_model: The embedding model to be used
  :return: a Chroma vectorstore
  """

  documents = []
  ids = []
  product_ids = products['product_id'].tolist() # Get list of product_id values from products_df
  for i, _ in enumerate(product_ids):
    product = products.iloc[i]
    #Create a document for each product, with its relevant details
    doc = Document(
            page_content=f"Product id: {product['product_id']}, Name: {product['name']}, Category: {product['category']}, Description: {product['description']}, Seasons: {product['seasons']}",
            metadata={
                "product_id": product['product_id'],
                "product_name": product['name'],
                "stock": product['stock'].item(),
                "price": product['price'].item(),
                "description": product['description'],
                }
            )
    documents.append(doc)
  chroma_db = Chroma.from_documents(documents=documents,
                                      embedding=embedding_model)
  return chroma_db

In [ ]:
products_db = index_product_catalog(products_df, embedding_model)

In [ ]:
def extract_products(emails: pd.DataFrame, store: Chroma, embed_model, k=4):
  """
  Extracts products refered to in the emails from the vector db.
  :param emails: Dataframe containing all the emails.
  :param store: the vector store containing the vectors from the catalog
  :param embed_model: the embedding model used
  :param k: number of possible relevant products to return
  """
  emails_dict = emails.to_dict(orient='records') if isinstance(emails, pd.DataFrame) else emails # Turn the email DataFrame into a dict, to make it usable in the prompt, after checking it is a DataFrame instance
  message_embeddings = [embed_model.embed_query(mail['message'].lower()) for mail in emails_dict] # turn messages into vectors
  extracted_documents = [store.similarity_search_by_vector(product_vector) for product_vector in message_embeddings] # extract products from store
  products_list = [{'product_id': document.metadata['product_id'], 'product_name': document.metadata['product_name'], 'stock': document.metadata['stock'], 'price': document.metadata['price'], 'description': document.metadata['description']} for document_list in extracted_documents for document in document_list] # present extracted products as a list of dicts
  return products_list

In [ ]:
def process_orders_and_handle_stock(emails, catalog):
  """
  Processes the orders and handles the stock, through gpt
  """
  prompt = (
      "You are a highly accurate stock assistant."
      "Check the availability of the products in the store's catalog against the orders received in the emails."
      "One email can contain orders for different items, in different quantities. Quantities can be explicit or implied."
      "Use all the details (name, id, category) in the email and in the catalog to find the requested item. Product details in the email may not respect case-sensitivity."
      "If the quantity requested is lower than the value of stock, create an order line with status as 'created'."
      "If it is unclear what the product requested is, use product_id 'AAA0000', otherwise use the product's ID and reduce stock accordingly."
      "If the quantity requested is higher thant the value of stock, create an order line with status as 'out of stock' and do not modify stock."
      "Please return only the output in the format : 'email_id::product_id::quantity::status'."
      "Email and product data will be supplied by user."
  )

  email_data = emails.to_dict(orient="records") if isinstance(emails, pd.DataFrame) else emails # Turn the email DataFrame into a dict, to make it usable in the prompt, after checking it is a DataFrame instance
  catalog_list = '\n'.join([f"ID: {product['product_id']}, Name: {product['product_name']}, Stock: {product['stock']}, Price: {product['price']}" for product in catalog]) # Flatten catalog, so that it can be used in the prompt

  # Send data to the ChatGPT API
  response = client.chat.completions.create(
      model=GPT_MODEL,
      messages=[
          {"role": "system", "content": prompt},
          {"role": "user", "content": f"Email Data:\n{email_data}"},
          {"role": "user", "content": f"Catalog:\n{catalog_list}"}
          ],
      temperature=0
      )

  returned_data = response.choices[0].message.content.strip().splitlines() # Clean response content and split lines
  cleaned_data = [entry.strip() for entry in returned_data if entry] # Clean each line

  return cleaned_data # Return cleaned data



In [ ]:
order_request_email_id = classifications.query("category == 'order request'")["Email ID"].values.tolist() # Select ids of emails classified as order request
order_request_emails = emails_df[emails_df['email_id'].isin(order_request_email_id)] # Only get order request emails
catalog = extract_products(order_request_emails, products_db, embedding_model) # product catalog
order_request_stocks = process_orders_and_handle_stock(order_request_emails, catalog)

In [ ]:
order_status = pd.DataFrame([entry.split('::') for entry in order_request_stocks], columns=["Email ID", "product ID", "quantity", "status"]) # Lay out the data for order-status
order_status_sheet.update([order_status.columns.values.tolist()] + order_status.values.tolist()) # Update Sheet with required resutls

{'spreadsheetId': '1K9yuCpqTY5lHlWHmq1ui2Ah7PZznHlWz4tqnbZgvndY',
 'updatedRange': "'order-status'!A1:D15",
 'updatedRows': 15,
 'updatedColumns': 4,
 'updatedCells': 60}

In [ ]:
def generate_order_request_responses(emails, order_status, catalog):
  """
  Generates order request responses through GPT
  """

  email_data = emails.to_dict(orient='records') if isinstance(emails, pd.DataFrame) else emails # Turn the email DataFrame into a dict, to make it usable in the prompt, after checking it is a DataFrame instance
  order_status_data = order_status.to_dict(orient='records')

  catalog_list = '\n'.join([f"ID: {product['product_id']}, Name: {product['product_name']}, Stock: {product['stock']}, Price: {product['price']}" for product in catalog]) # Flatten catalog, so that it can be used in the prompt

  prompt = (
        "You are a polite, helpful and professional sales assistant in our online fashion store speaking as a member of the customer service."
        "For each of the order lines, you will write a production-ready email response with a professional layout, using a professional tone, avoiding contractions."
        "If the customer's name is in the email, use it in your response, otherwise use a generic yet relevant and professional term to address them."
        "If the the status of the order is 'created' and quantity is greater than 0,the order is fully processed and you will inform and provide product details, such as its name, ID, price and total price for the order."
        "If the status of the order is 'out of stock', the order cannot be fully processed and you will explain the situation and suggest alternatives or options, like waiting for restock."
        "If the item could not be identified precisely (product ID AAA0000), consider the order cannot be processed, do not say it is out of stock and ask for more precisions, offer alternatives or ask the customer to confirm or infirm your guesses about the product."
        "If the customer wants to wait and place the order later, offer to contact them at the time they consider placing the order, if known."
        "The email signature shall not mention a name, only the service in which you work."
        "Provide the output only in the format: 'email_id::response_email' and separate each entry with '-----'."
        "Email, product and order status data will be supplied by user."
  )

  # Send data to the ChatGPT API
  response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"\n\nEmail Data:\n{email_data}"},
            {"role": "user", "content": f"\n\nOrder Status Data:\n{order_status_data}"},
            {"role": "user", "content": f"\n\nCatalog Data:\n{catalog_list}"}
        ]
    )

  returned_data = response.choices[0].message.content.strip().split('-----') # Clean response content and split lines along separator
  cleaned_data = [entry.strip() for entry in returned_data if entry] # Clean each line
  return cleaned_data

In [ ]:
order_status_responses = generate_order_request_responses(order_request_emails, order_status, catalog)
order_status_responses_df = pd.DataFrame([entry.split('::') for entry in order_status_responses], columns=["Email ID", "response"]) # Lay out the data for order-status
order_response_sheet.update([order_status_responses_df.columns.values.tolist()] + order_status_responses_df.values.tolist()) # Update Sheet with required resutls

{'spreadsheetId': '1K9yuCpqTY5lHlWHmq1ui2Ah7PZznHlWz4tqnbZgvndY',
 'updatedRange': "'order-response'!A1:B16",
 'updatedRows': 16,
 'updatedColumns': 2,
 'updatedCells': 32}

# Task 3. Handle product inquiry

In [ ]:
inquiry_email_id = classifications.query("category == 'product inquiry'")["Email ID"].values.tolist() # Select ids of emails classified as inquiries
inquiry_emails = emails_df[emails_df['email_id'].isin(inquiry_email_id)] # Only get inquiry emails

In [ ]:
def generate_inquiry_responses(emails, catalog):
  """
  Generate inquiry responses through GPT
  """

  email_data = emails.to_dict(orient='records') if isinstance(emails, pd.DataFrame) else emails # Turn the email DataFrame into a dict, to make it usable in the prompt, after checking it is a DataFrame instance
  catalog_list = '\n'.join([f"ID: {product['product_id']}, Name: {product['product_name']}, Stock: {product['stock']}, Price: {product['price']}" for product in catalog]) # Flatten catalog

  prompt = (
        "You are a polite, helpful and professional sales assistant in our online fashion store speaking as a member of the customer service."
        "For each of the product inquiries, you will write a production-ready email response with a professional layout, using a professional tone, avoiding contractions, and using relevant information from the product catalog."
        "You will pay attention to every aspect of the products asked about by the customer, or that you advise, like the season it is to be used in and the category it belongs to."
        "If the customer's name is in the inquiry email, use it in your response, otherwise use a generic, yet relevant and professional term to address them."
        "The email signature shall not mention a name, only the service in which you work."
        "Provide the output only in the format: 'email_id::response_email' and separate each entry with '-----'."
        "Email and product data will be supplied by user."
  )

  response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"\n\nEmail Data:\n{email_data}"},
            {"role": "user", "content": f"\n\nCatalog Data:\n{catalog_list}"}
        ]
    )

  returned_data = response.choices[0].message.content.strip().split('-----') # Clean response content and split lines along separator
  return returned_data

In [ ]:
inquiry_responses = generate_inquiry_responses(inquiry_emails, catalog)
inquiry_responses_df = pd.DataFrame([entry.split('::') for entry in inquiry_responses], columns=["Email ID", "response"]) # Lay out the data for order-status
inquiry_response_sheet.update([inquiry_responses_df.columns.values.tolist()] + inquiry_responses_df.values.tolist()) # Update Sheet with required resutls

{'spreadsheetId': '1K9yuCpqTY5lHlWHmq1ui2Ah7PZznHlWz4tqnbZgvndY',
 'updatedRange': "'inquiry-response'!A1:B10",
 'updatedRows': 10,
 'updatedColumns': 2,
 'updatedCells': 20}

# More elements on the choices I have made

I have chosen to extract all products for all emails in one pass, in order to make the code more concise and - hopefully - faster and more efficient.<br>
Depending on how emails are received and pre-processed, processing each email one after the other, extracting only the products for each email everytime could be possible. I simply went on to the easiest way to deal with the data I was given and have decided to process the emails in batches.<br>
Looping through a DataFrame being anything but optimized, I have chosen to resort to list comprehensions, as DataFrames could not be used directly in the prompt, in order to speed the process up a bit.<br>
I chose to use an arbitrary product id for products that cannot be figured out by the LLM, and I made sure this value was non existent AND I chose a value that was unlikely to exist in the future. Of course, this value could be changed at any moment in time if need be.